# COGS 118A Final Project
Binh Nguyen

In [6]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import string
import csv
%matplotlib inline

Datasets acquired from UCI Machine Learning Repository:
https://archive.ics.uci.edu/ml/datasets

## Load datasets


### 1. LETTER Dataset
https://archive.ics.uci.edu/ml/datasets/letter+recognition

In [18]:
# Size = 20000x17
# N = 20000
# D = 16 features
data = np.genfromtxt('letter-recognition.data', delimiter = ',', dtype=None, encoding=None)
n = np.size(data)
d = 16
X = np.zeros((n,d))

# Create X Matrix from numpy.void type
for i in range(n):
    for j in range(d):
        X[i][j] = data[i][j+1]
        
# Create y vector with labels for each letter
y = []
for i in range(n):
    y.append(data[i][0])

# Convert letters to positive and negative values

# A-M as positive (+) = 1
positive = list(string.ascii_uppercase)[0:13]

# N-Z as negative (-) = 0
negative = list(string.ascii_uppercase)[13:26]

y = np.asarray(y)

for i in range(len(y)):
    for j in range(len(positive)):
        if y[i] == positive[j]:
            y[i] = 1
        elif y[i] == negative[j]:
            y[i] = 0

y = y.astype(int)